## Speed up Pandas' GroupBy with Cython

Scenario: we have a large dataframe [comprising](https://en.wikipedia.org/wiki/User:Giraffedata/comprised_of) several smaller tables concatenated together.

We've done all our data cleaning and munging on the columns of the large dataframe, and we need to break it up into small dataframes.  This scenario may, hypothetically, come up when doing sequence modeling with recurrent neural networks, and we need to prepare mini-batches of sequences for our model to train against.

The obvious way to do it is `list(df.groupby(level=0, sort=False))`.  Unfortunately, Pandas can be really slow in this case -- making lots of little dataframes is an expensive operation.

Since we can make certain guarantees of our data, can we speed this up?

In [1]:
import pandas as pd
import numpy as np
from uuid import uuid4
import toolz

In [2]:
%load_ext Cython

In [43]:
nblocks = 10000
rowsperblock = np.random.randint(1, 7, size=nblocks)
ncols = 3

idx = list(toolz.concat([[uuid4().hex] * rpb for rpb in rowsperblock]))
df = pd.DataFrame(np.random.standard_normal(size=(rowsperblock.sum(), ncols)), index=idx)
print(df.info())
df.head(15)

<class 'pandas.core.frame.DataFrame'>
Index: 35175 entries, 1c251773cd9b47e4844c87fa5510e4f9 to b32ba494106e494f9361497b50765e6e
Data columns (total 3 columns):
0    35175 non-null float64
1    35175 non-null float64
2    35175 non-null float64
dtypes: float64(3)
memory usage: 1.1+ MB
None


,0,1,2
1c251773cd9b47e4844c87fa5510e4f9,-0.486437,-1.347221,1.471057
1c251773cd9b47e4844c87fa5510e4f9,-0.443606,-1.118939,-1.255591
ac09fc3812a941d694610e5ce162a46c,0.224710,1.019528,0.546751
ac09fc3812a941d694610e5ce162a46c,0.546464,-1.223292,1.430216
ac09fc3812a941d694610e5ce162a46c,-0.776167,1.373449,0.848901
ac09fc3812a941d694610e5ce162a46c,-0.353192,1.203521,-0.486421
ac09fc3812a941d694610e5ce162a46c,0.356607,1.921953,1.645956
30dc2004ba3c492c9ad93e263411f0f4,0.053396,1.382598,-0.767970
30dc2004ba3c492c9ad93e263411f0f4,-0.293145,0.508521,-0.191436
30dc2004ba3c492c9ad93e263411f0f4,-2.679478,0.357335,0.178980


In [44]:
df.index.is_monotonic

False

In [45]:
list(df.groupby(level=[0], sort=False))[:2]

[('1c251773cd9b47e4844c87fa5510e4f9',
                                           0         1         2
  1c251773cd9b47e4844c87fa5510e4f9 -0.486437 -1.347221  1.471057
  1c251773cd9b47e4844c87fa5510e4f9 -0.443606 -1.118939 -1.255591),
 ('ac09fc3812a941d694610e5ce162a46c',
                                           0         1         2
  ac09fc3812a941d694610e5ce162a46c  0.224710  1.019528  0.546751
  ac09fc3812a941d694610e5ce162a46c  0.546464 -1.223292  1.430216
  ac09fc3812a941d694610e5ce162a46c -0.776167  1.373449  0.848901
  ac09fc3812a941d694610e5ce162a46c -0.353192  1.203521 -0.486421
  ac09fc3812a941d694610e5ce162a46c  0.356607  1.921953  1.645956)]

In [46]:
%timeit list(df.groupby(level=[0], sort=False))

821 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%prun list(df.groupby(level=[0], sort=False))

### Creating lots of little dataframes is expensive
* Pandas doesn't have an optimized fast path for data that's already clean.

## Speedup version 1: use NumPy

* For this problem, we're willing to adjust the output and generate a sequence of NumPy arrays rather than a sequence of dataframes.
* With this adjustment, we'll see we can get a substantial speedup using NumPy operations.
* Once it's rewritten to use NumPy, then we can get *further* speedups with Cython.

In [47]:
def splitby(df):
    idx = df.index
    # NumPy array of "posts" that delineate the row indices
    # with which to split the dataframe.
    posts = np.where(idx[1:] != idx[:-1])[0] + 1
    split_labels = idx[np.concatenate([[0], posts, [-1]])]
    split_data = np.split(df.values, posts, 0)
    return list(zip(split_labels, split_data))

In [48]:
splitby(df)[:2]

[('1c251773cd9b47e4844c87fa5510e4f9',
  array([[-0.48643698, -1.34722094,  1.47105721],
         [-0.44360622, -1.11893904, -1.25559088]])),
 ('ac09fc3812a941d694610e5ce162a46c',
  array([[ 0.2247096 ,  1.01952771,  0.54675057],
         [ 0.54646353, -1.22329178,  1.43021627],
         [-0.77616727,  1.37344933,  0.84890117],
         [-0.35319165,  1.2035212 , -0.48642063],
         [ 0.35660683,  1.92195255,  1.64595556]]))]

In [49]:
%timeit splitby(df)

16.9 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Questions
* Why is this faster?
* If we double the number of columns, how do you expect the two versions to scale?

## Speedup version 2: Cython

In [55]:
%%cython -a

cimport cython
import numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def splitby_cython(df):
    cdef:
        double[:,:] cols = df.values
        object[:] idx = df.index.values
        int n = idx.shape[0]
        list result = []
        int i, thispost = 0
        
    for i in range(1, n):
        if idx[i] != idx[i-1]:
            result.append((idx[i-1], cols[thispost:i]))
            thispost = i
    result.append((idx[i-1], cols[thispost:]))
    return result

In [57]:
%timeit splitby_cython(df)

4.54 ms ± 85.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
